## parse

Sources: https://www.kaggle.com/rmisra/news-category-dataset
Reads the json file properly

In [1]:
import json

with open('2_news_category_encoded_11000.json') as file:
    news = file.read()

In [ ]:
news_list = news.split('\n')

dict_list = list()
for str_dict in range(len(news_list)):
    try:
        current_dict = json.loads(news_list[str_dict])
        current_dict['_id'] = str_dict
        dict_list.append(current_dict)
    except:
        pass
dict_list

## encode as json + import

Encode and bulk insert at the same time

In [14]:
import relevanceai

project = "michelangioloma"
api_key = "TDFvdm9Yd0J1M3VVTEcyQTV3VWs6b3RnRXJnUW9SQ2kwMVVJWVZ2VzVtZw"
client = relevanceai.Client(project, api_key)

In [ ]:
#encode on local
from vectorhub.encoders.text.sentence_transformers import SentenceTransformer2Vec
import progressbar
from time import sleep

dict_list_ = dict_list.copy()

bar = progressbar.ProgressBar(maxval=len(dict_list_), widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
model = SentenceTransformer2Vec("all-MiniLM-L6-v2")

dict_list_final = list()
jump = 1000
bar.start()
for iter in range(jump, len(dict_list_), jump):
    bar.update(iter)
    current_vectors = model.encode_documents(documents=dict_list_[iter-jump:iter], fields=['short_description'])
    client.datasets.bulk_insert(dataset_id='news_category_dataset', documents=current_vectors)
    dict_list_final += current_vectors
bar.finish()

#missing elements
current_vectors = model.encode_documents(documents=dict_list_[iter:], fields=['short_description'])
client.datasets.bulk_insert(dataset_id='news_category_dataset', documents=current_vectors)
dict_list_final += current_vectors
#save json to local

In [ ]:
a_file = open("news_category_dataset_encoded_1.json", "w")
json.dump(dict_list_final, a_file)
a_file.close()

In [32]:
client.datasets.schema('news_category_dataset')

{'authors': 'text',
 'category': 'text',
 'date': 'date',
 'headline': 'text',
 'insert_date_': 'date',
 'link': 'text',
 'short_description': 'text',
 'short_description_sentence_transformers_vector_': {'vector': 384}}

### store only 100MB

The encoded data (50% of the original) si 1GB, I need to save 100MB only to upload a df to GitHub

In [8]:
import json
with open('2_news_category_encoded_11000.json') as file:
    news_dict = json.load(file)
len(news_dict)

11000

In [7]:

a_file = open("news_category_dataset_encoded_little.json", "w")
json.dump(news_dict[0:11000], a_file)
a_file.close()

## encode as csv + import

In [37]:
import pandas as pd

df = pd.DataFrame(dict_list)[['category', 'short_description']]
df = df.reset_index()
df

,index,category,short_description
0,0,CRIME,She left her husband. He killed their children...
1,1,ENTERTAINMENT,Of course it has a song.
2,2,ENTERTAINMENT,The actor and his longtime girlfriend Anna Ebe...
3,3,ENTERTAINMENT,The actor gives Dems an ass-kicking for not fi...
4,4,ENTERTAINMENT,"The ""Dietland"" actress said using the bags is ..."
...,...,...,...
200848,200848,TECH,Verizon Wireless and AT&T are already promotin...
200849,200849,SPORTS,"Afterward, Azarenka, more effusive with the pr..."
200850,200850,SPORTS,"Leading up to Super Bowl XLVI, the most talked..."
200851,200851,SPORTS,CORRECTION: An earlier version of this story i...


In [38]:
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import numpy as np

tqdm.pandas()

model = SentenceTransformer('all-MiniLM-L6-v2') #all-MiniLM-L6-v2 #all-mpnet-base-v2

def encode_desc(df, index_col, description_col):
    df_ = df.copy()
    df_[description_col] = df_[description_col].progress_apply(lambda x : model.encode(x))
    df_index = df_.pop(index_col)
    df_ = df_[[description_col]]
    df_ = pd.DataFrame(np.column_stack(list(zip(*df_.values))))
    df_.index = df_index
    return df_

df = encode_desc(df, 'index', 'short_description')
df.to_csv('encoded_articles.csv')
df

100%|██████████| 200853/200853 [55:21<00:00, 60.47it/s]  


,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
index,,,,,,,,,,,,,,,,,,,,,
0,-0.002307,-0.042142,0.000844,0.084845,0.058054,0.024377,0.005735,-0.009646,0.009666,-0.027164,...,0.031225,-0.061444,0.010794,-0.048192,-0.052593,0.051760,-0.041878,0.001470,0.065105,-0.003955
1,-0.084582,-0.059715,-0.023733,-0.043918,0.004473,0.044030,0.029373,-0.109548,0.030384,-0.012632,...,0.004259,0.058051,-0.039785,0.027936,0.032938,-0.084325,0.028954,0.021029,0.117550,-0.020887
2,-0.013749,0.042038,0.017863,0.025209,-0.033090,0.068688,0.078486,0.040208,0.060893,0.008566,...,-0.045137,-0.082507,0.039340,0.043492,0.083472,0.027767,-0.030879,-0.071492,-0.020391,-0.043173
3,-0.004410,0.028771,0.020945,0.006848,0.008203,0.023190,0.037188,-0.034391,-0.001100,-0.038336,...,0.025089,-0.003878,-0.007339,-0.030097,0.027124,0.023840,0.003165,-0.029565,0.079411,0.016206
4,0.020985,0.059220,-0.031126,0.041178,0.014413,-0.017519,0.156668,-0.002552,0.011420,-0.074701,...,-0.005784,-0.002244,-0.001376,-0.020889,0.045921,0.053874,0.018058,0.010689,-0.008190,-0.046759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200848,-0.073026,-0.054238,0.057068,-0.093490,0.016852,0.027932,-0.009719,0.040270,0.045707,-0.022895,...,0.007804,-0.019020,0.063627,-0.011037,-0.012503,0.008248,-0.007223,-0.013183,0.125019,0.044208
200849,-0.007025,0.028488,-0.034769,0.022991,-0.015518,0.074670,0.055721,0.099934,0.052673,0.062930,...,0.007293,-0.024251,-0.054619,-0.017861,-0.065881,0.049066,-0.017081,-0.028843,-0.044344,-0.000740
200850,-0.006769,-0.013464,0.018895,-0.090075,0.033060,0.044755,-0.062763,-0.016087,0.073637,-0.007146,...,0.114696,0.040547,-0.023632,0.014679,0.003725,-0.053574,0.025160,0.104872,0.015064,0.078087


In [41]:
df = df.reset_index()
df

,index,0,1,2,3,4,5,6,7,8,...,374,375,376,377,378,379,380,381,382,383
0,0,-0.002307,-0.042142,0.000844,0.084845,0.058054,0.024377,0.005735,-0.009646,0.009666,...,0.031225,-0.061444,0.010794,-0.048192,-0.052593,0.051760,-0.041878,0.001470,0.065105,-0.003955
1,1,-0.084582,-0.059715,-0.023733,-0.043918,0.004473,0.044030,0.029373,-0.109548,0.030384,...,0.004259,0.058051,-0.039785,0.027936,0.032938,-0.084325,0.028954,0.021029,0.117550,-0.020887
2,2,-0.013749,0.042038,0.017863,0.025209,-0.033090,0.068688,0.078486,0.040208,0.060893,...,-0.045137,-0.082507,0.039340,0.043492,0.083472,0.027767,-0.030879,-0.071492,-0.020391,-0.043173
3,3,-0.004410,0.028771,0.020945,0.006848,0.008203,0.023190,0.037188,-0.034391,-0.001100,...,0.025089,-0.003878,-0.007339,-0.030097,0.027124,0.023840,0.003165,-0.029565,0.079411,0.016206
4,4,0.020985,0.059220,-0.031126,0.041178,0.014413,-0.017519,0.156668,-0.002552,0.011420,...,-0.005784,-0.002244,-0.001376,-0.020889,0.045921,0.053874,0.018058,0.010689,-0.008190,-0.046759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200848,200848,-0.073026,-0.054238,0.057068,-0.093490,0.016852,0.027932,-0.009719,0.040270,0.045707,...,0.007804,-0.019020,0.063627,-0.011037,-0.012503,0.008248,-0.007223,-0.013183,0.125019,0.044208
200849,200849,-0.007025,0.028488,-0.034769,0.022991,-0.015518,0.074670,0.055721,0.099934,0.052673,...,0.007293,-0.024251,-0.054619,-0.017861,-0.065881,0.049066,-0.017081,-0.028843,-0.044344,-0.000740
200850,200850,-0.006769,-0.013464,0.018895,-0.090075,0.033060,0.044755,-0.062763,-0.016087,0.073637,...,0.114696,0.040547,-0.023632,0.014679,0.003725,-0.053574,0.025160,0.104872,0.015064,0.078087
200851,200851,-0.013976,0.041705,-0.027639,0.009262,0.012049,0.117540,-0.024201,-0.020186,0.012517,...,0.029007,-0.038343,-0.101093,-0.065580,-0.025972,0.016242,-0.042031,0.010189,-0.000396,0.061747


In [ ]:
df.to_json(orient='records')

In [ ]:
list1 = list()
for index in range(len(df['index'])):
    if index % 100 == 0:
        print(index)
    dict1 = dict()
    vectors = list(df.drop('index', axis=1).values[index])
    dict1['_id'] = index
    dict1['movie_transformers_vector_'] = vectors
    list1.append(dict1)
list1

### relevanceai

IMPOSSIBLE TO MAKE, I am trying to convert the csv to json before uploading to relevanceai, take 6hrs

In [8]:
import relevanceai

project = "michelangioloma"
api_key = "TDFvdm9Yd0J1M3VVTEcyQTV3VWs6b3RnRXJnUW9SQ2kwMVVJWVZ2VzVtZw"
client = relevanceai.Client(project, api_key)

In [ ]:
list1 = list()
for index in range(len(df['title'])):
    if index % 100 == 0:
        print(index)
    dict1 = dict()
    title = df['title'].iloc[index]
    vectors = list(df.drop('title', axis=1).values[index])
    dict1['_id'] = index
    dict1['movie_transformers_vector_'] = vectors
    list1.append(dict1)
list1

client.datasets.bulk_insert(dataset_id='mpst_movie_dataset', documents=list1)